## Sample analysis of ecommerce CRM

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Database had two tables with sales and returns data
Sales = pd.read_csv('Sales.csv')
Returns = pd.read_csv('Returns.csv')

In [3]:
Sales.head()

,Client_id,Transaction_id,Date,Products_count,SKU,Net_value,Gross_value
0,2530298,405832,2015-01-20 09:18:22,7,7,42.76,52.59
1,2348582,405833,2015-01-20 09:30:26,3,3,23.26,28.61
2,2754973,405834,2015-01-20 09:30:39,2,2,16.99,20.90
3,2751501,405835,2015-01-20 09:42:47,13,8,57.02,70.13
4,2334605,405836,2015-01-20 09:43:56,4,4,24.87,30.59


In [4]:
Returns.head()

,Transaction_id,Return_id,Returned_products_count,Returns_net_value
0,1071769,31687,10,21.12
1,768981,31688,1,8.40
2,1099412,31689,8,22.70
3,576370,31690,9,46.28
4,1048352,31691,1,17.90


In [5]:
# size
Sales.shape, Returns.shape

((800000, 7), (93454, 4))

## Sample queries

How many clients placed more than 5 orders within given period ?

In [6]:
# count transactions per client
temp = Sales.groupby(['Client_id'], as_index = False)['Transaction_id'].count()
# ename column to count
temp.rename(columns = {'Transaction_id':'count'}, inplace = True)
# ilter rows with count >5 and count these rows
temp[temp['count'] >5].count()

Client_id    2474
count        2474
dtype: int64

What is the average count of returned products as per 100.000 clients 
 (we take into consideration those transactions that had some returns)

In [7]:
# filter Sales table by transactions that had returns
SalesWithReturns= pd.merge(Sales, Returns, on = 'Transaction_id', how = 'right')
#SalesWithReturns.head()

In [8]:
# sum of returned products
SalesWithReturns.Returned_products_count.sum()

342548

In [9]:
# Numer of unique Clients
SalesWithReturns.Client_id.nunique()

85713

In [10]:
# average number of returned products per 100000 unique clients
SalesWithReturns.Returned_products_count.sum()/ SalesWithReturns.Client_id.nunique()* 100000

399645.3280132535

What is the gross value of orders made by top 50 customers. How this amount will change if you take returns into consideration

In [11]:
#merge with Sales with Returns to calculate sales after returns
temp = pd.merge(Sales, Returns, on = 'Transaction_id', how = 'left')
temp.head()
# fill Nan in 'Returns_net_value' column, i.e. where there were no returns
temp.Returns_net_value.fillna(0, inplace = True)
# gross value with returns
temp['Gross_value_after_ret'] = temp['Gross_value'] - temp['Returns_net_value']*1.23
temp.head()

,Client_id,Transaction_id,Date,Products_count,SKU,Net_value,Gross_value,Return_id,Returned_products_count,Returns_net_value,Gross_value_after_ret
0,2530298,405832,2015-01-20 09:18:22,7,7,42.76,52.59,NaN,NaN,0.00,52.5900
1,2348582,405833,2015-01-20 09:30:26,3,3,23.26,28.61,73903.0,2.0,11.63,14.3051
2,2754973,405834,2015-01-20 09:30:39,2,2,16.99,20.90,NaN,NaN,0.00,20.9000
3,2751501,405835,2015-01-20 09:42:47,13,8,57.02,70.13,NaN,NaN,0.00,70.1300
4,2334605,405836,2015-01-20 09:43:56,4,4,24.87,30.59,NaN,NaN,0.00,30.5900


In [12]:
#calculate sales value per Client_id
temp2 = temp.groupby(['Client_id'], as_index = False)[['Gross_value', 'Gross_value_after_ret']].sum()
#sort descending by 'Gross_value' before returns !
temp3 = temp2.sort_values('Gross_value', ascending = False)
# filter top 50 
temp4 = temp3[0:50]

In [13]:
#sum of the top 50
temp4['Gross_value'].sum(), temp4['Gross_value_after_ret'].sum()

(17217.390000000003, 15903.934500000001)

In how many orders there are more products than product types (i.e. SKUs) after including returns

In [14]:
#merge Sales and Returns table
temp = pd.merge(Sales, Returns, on = 'Transaction_id', how = 'left')
# fill Nan in 'Returns_products_count' column, i.e. where there were no returns
temp.Returned_products_count.fillna(0, inplace = True)

In [15]:
temp.head()

,Client_id,Transaction_id,Date,Products_count,SKU,Net_value,Gross_value,Return_id,Returned_products_count,Returns_net_value
0,2530298,405832,2015-01-20 09:18:22,7,7,42.76,52.59,NaN,0.0,NaN
1,2348582,405833,2015-01-20 09:30:26,3,3,23.26,28.61,73903.0,2.0,11.63
2,2754973,405834,2015-01-20 09:30:39,2,2,16.99,20.90,NaN,0.0,NaN
3,2751501,405835,2015-01-20 09:42:47,13,8,57.02,70.13,NaN,0.0,NaN
4,2334605,405836,2015-01-20 09:43:56,4,4,24.87,30.59,NaN,0.0,NaN


In [16]:
# calculate products after returns
temp['Products_count_after_returns'] = temp['Products_count'] - temp['Returned_products_count']

In [17]:
# filter the condition
temp2 = temp[temp['Products_count_after_returns'] > temp['SKU']]
# calculate rows filtered rows
len(temp2)

76460

In [18]:
# group transactions in case there are duplicates
temp3 = temp2.groupby(['Transaction_id'])['Products_count_after_returns'].sum()
len(temp3)

76460

What is the average order net value

In [19]:
# calculate net_value per transaction, in case there are duplicates
temp = Sales.groupby(['Transaction_id'], as_index = False)['Net_value'].sum()
# calculate mean
temp['Net_value'].mean()

29.50451636249986

What is the average order net value after returns

In [20]:
#merge with Sales with Returns to calculate sales after returns
temp = pd.merge(Sales, Returns, on = 'Transaction_id', how = 'left')
temp.head()
# fill Nan in 'Returns_net_value' column, i.e. where there were no returns
temp.Returns_net_value.fillna(0, inplace = True)
# gross value with returns
temp['Net_value_after_ret'] = temp['Net_value'] - temp['Returns_net_value']
#temp.head()

In [21]:
# calculate net_value per transaction, in case there are duplicates
temp2 = temp.groupby(['Transaction_id'], as_index = False)['Net_value_after_ret'].sum()
# calculate mean
temp2['Net_value_after_ret'].mean()

27.303006412500086

How the value and amount of returns changed over the years

In [22]:
# join Sales to Retuns to add date
temp = pd.merge(Sales, Returns, on = 'Transaction_id', how = 'right')
temp.head()

,Client_id,Transaction_id,Date,Products_count,SKU,Net_value,Gross_value,Return_id,Returned_products_count,Returns_net_value
0,2491562,1071769,2019-05-25 15:30:36,11,6,42.24,51.96,31687,10,21.12
1,2462789,768981,2017-06-04 03:24:44,1,1,8.40,10.33,31688,1,8.40
2,2487287,1099412,2019-07-31 01:52:18,10,10,45.41,55.85,31689,8,22.70
3,2479812,576370,2016-03-03 00:58:43,9,5,46.28,56.92,31690,9,46.28
4,2572558,1048352,2019-03-30 13:49:06,1,1,17.90,22.02,31691,1,17.90


In [23]:
#convert to datetime
temp['Date2'] = pd.to_datetime(temp['Date'])
# extract year
temp['Year'] = temp['Date2'].dt.year
# groupby year
temp2 = temp.groupby(temp['Year'])[['Returns_net_value', 'Returned_products_count']].sum()

In [24]:
temp2

,Returns_net_value,Returned_products_count
Year,,
2015,317818.80,62191
2016,339755.88,66249
2017,335833.94,65859
2018,344788.24,66312
2019,331093.03,63898
2020,92376.05,18039


Which client had the biggest gap between orders

In [25]:
# create a copy
Sales2 = Sales.copy(deep = True)
#fix error
Sales2['Date'].min()

'100-12-12 00:00:00'

In [26]:
# find the index of wrong row
Sales2[Sales2['Date'] ==Sales2['Date'].min()]

,Client_id,Transaction_id,Date,Products_count,SKU,Net_value,Gross_value
761679,2387523,1167511,100-12-12 00:00:00,6,6,36.1,44.4


In [27]:
# drop row with wrong date
Sales2.drop(index = 761679, inplace = True)

In [28]:
#check
Sales2.shape, Sales2['Date'].min()

((799999, 7), '10:55:06 02-03-2019')

In [29]:
#fix second error
Sales2[Sales2['Date'] ==Sales2['Date'].min()]

,Client_id,Transaction_id,Date,Products_count,SKU,Net_value,Gross_value
630677,2459730,1036509,10:55:06 02-03-2019,4,4,27.96,34.39


In [30]:
# drop row with wrong date
Sales2.drop(index = 630677, inplace = True)

In [31]:
# drop unecessary columns
Sales2.drop(['Transaction_id', 'Products_count', 'SKU', 'Net_value','Gross_value'], axis = 1, inplace = True)
# sort
Sales2 = Sales2.sort_values(by = ['Client_id','Date'])

In [32]:
#convert to datetime
Sales2['Date'] = pd.to_datetime(Sales2['Date'])
# create temp columns with values of next row
Sales2['Client_temp'] =  Sales2['Client_id'].shift(-1)
Sales2['Date_temp'] =  Sales2['Date'].shift(-1)

In [33]:
# calculate the date difference in case next row has data of the same client
Sales2['datediff'] = np.where(Sales2['Client_temp'] ==Sales2['Client_id'], Sales2['Date_temp'] - Sales2['Date'], 0)

In [34]:
Sales2.head(10)

,Client_id,Date,Client_temp,Date_temp,datediff
318195,2236822,2017-02-16 16:40:20,2236823.0,2016-04-27 23:47:24,0 days 00:00:00
194179,2236823,2016-04-27 23:47:24,2236823.0,2017-03-31 03:53:43,337 days 04:06:19
335876,2236823,2017-03-31 03:53:43,2236823.0,2017-04-06 00:23:33,5 days 20:29:50
338272,2236823,2017-04-06 00:23:33,2236823.0,2018-10-16 04:55:31,558 days 04:31:58
572887,2236823,2018-10-16 04:55:31,2236824.0,2018-08-28 21:25:21,0 days 00:00:00
552681,2236824,2018-08-28 21:25:21,2236825.0,2017-05-19 12:35:11,0 days 00:00:00
356521,2236825,2017-05-19 12:35:11,2236825.0,2018-01-07 16:40:34,233 days 04:05:23
454476,2236825,2018-01-07 16:40:34,2236826.0,2015-07-13 15:36:58,0 days 00:00:00
72903,2236826,2015-07-13 15:36:58,2236826.0,2016-11-05 02:50:20,480 days 11:13:22
274519,2236826,2016-11-05 02:50:20,2236827.0,2015-07-03 14:50:06,0 days 00:00:00


In [35]:
Sales2[Sales2['datediff']==Sales2['datediff'].max()]

,Client_id,Date,Client_temp,Date_temp,datediff
341,2700022,2015-01-21 04:20:33,2700022.0,2020-04-02 21:59:00,1898 days 17:38:27


In [36]:
Sales.head()

,Client_id,Transaction_id,Date,Products_count,SKU,Net_value,Gross_value
0,2530298,405832,2015-01-20 09:18:22,7,7,42.76,52.59
1,2348582,405833,2015-01-20 09:30:26,3,3,23.26,28.61
2,2754973,405834,2015-01-20 09:30:39,2,2,16.99,20.90
3,2751501,405835,2015-01-20 09:42:47,13,8,57.02,70.13
4,2334605,405836,2015-01-20 09:43:56,4,4,24.87,30.59


Summary of the top 100 clients

In [37]:
#merge with Sales with Returns to calculate sales after returns
temp = pd.merge(Sales, Returns, on = 'Transaction_id', how = 'left')

In [38]:
temp.Returned_products_count.fillna(0, inplace = True)
temp.Returns_net_value.fillna(0, inplace = True)

In [39]:
# calculate net value after returns
temp['Net_value_after_ret'] = temp['Net_value'] - temp['Returns_net_value']
# calculate products after returns
temp['Products_count_after_returns'] = temp['Products_count'] - temp['Returned_products_count']

In [40]:
temp.head()

,Client_id,Transaction_id,Date,Products_count,SKU,Net_value,Gross_value,Return_id,Returned_products_count,Returns_net_value,Net_value_after_ret,Products_count_after_returns
0,2530298,405832,2015-01-20 09:18:22,7,7,42.76,52.59,NaN,0.0,0.00,42.76,7.0
1,2348582,405833,2015-01-20 09:30:26,3,3,23.26,28.61,73903.0,2.0,11.63,11.63,1.0
2,2754973,405834,2015-01-20 09:30:39,2,2,16.99,20.90,NaN,0.0,0.00,16.99,2.0
3,2751501,405835,2015-01-20 09:42:47,13,8,57.02,70.13,NaN,0.0,0.00,57.02,13.0
4,2334605,405836,2015-01-20 09:43:56,4,4,24.87,30.59,NaN,0.0,0.00,24.87,4.0


In [41]:
temp2 = temp.groupby('Client_id').agg(
    transaction_count = ('Transaction_id', 'count'),
    first_order = ('Date', 'min'),
    last_order = ('Date', 'max'),
    products_count = ('Products_count', 'sum'),
    net_value = ('Net_value', 'sum'),
    products_count_after_returns = ('Products_count_after_returns', 'sum'),
    net_value_after_returns = ('Net_value_after_ret', 'sum')
   )

In [42]:
temp2.head()

,transaction_count,first_order,last_order,products_count,net_value,products_count_after_returns,net_value_after_returns
Client_id,,,,,,,
2236822,1,2017-02-16 16:40:20,2017-02-16 16:40:20,3,34.78,3.0,34.78
2236823,4,2016-04-27 23:47:24,2018-10-16 04:55:31,10,143.67,10.0,143.67
2236824,1,2018-08-28 21:25:21,2018-08-28 21:25:21,3,35.11,3.0,35.11
2236825,2,2017-05-19 12:35:11,2018-01-07 16:40:34,7,52.55,7.0,52.55
2236826,2,2015-07-13 15:36:58,2016-11-05 02:50:20,9,47.96,9.0,47.96


In [43]:
temp3 = temp2.sort_values('net_value', ascending = False)
temp3.head(10)

,transaction_count,first_order,last_order,products_count,net_value,products_count_after_returns,net_value_after_returns
Client_id,,,,,,,
2744614,8,2015-03-03 11:30:15,2019-05-26 06:31:47,60,319.79,60.0,319.79
2746578,7,2015-05-31 01:25:37,2019-11-14 18:49:38,33,309.70,32.0,291.33
2311550,10,2015-06-14 11:02:15,2019-01-11 13:55:14,49,307.84,39.0,267.57
2249080,8,2016-01-05 15:51:50,2020-03-22 18:57:16,57,305.02,40.0,192.32
2668998,8,2016-02-25 21:16:43,2020-04-01 18:20:28,33,297.26,32.0,276.06
2647872,8,2015-07-30 14:00:44,2018-05-03 20:38:31,34,295.99,32.0,273.46
2684988,8,2015-05-07 21:23:12,2019-04-28 22:02:56,43,293.47,40.0,276.38
2403639,8,2015-10-23 17:08:11,2020-01-23 00:42:58,58,292.11,53.0,257.88
2656167,7,2015-02-24 15:53:45,2018-10-05 06:26:34,41,291.56,37.0,268.30
